In [14]:
import pandas as pd
import sys
import numpy as np
import os
import mysql.connector
from tqdm import tqdm
import re

In [78]:
path_model = os.path.join(os.getcwd(),'DATABASE_DP')
path_dwh = os.path.join(os.getcwd(),'..','3_data_preprocesing_module','DATAWAREHOUSE')
path_dwh_pd = os.path.join(os.getcwd(),'DATAWAREHOUSE_DP')

In [3]:
filelist = os.listdir(path_model)

['.ipynb_checkpoints',
 'ci_cp_num_cap.csv',
 'ci_cp_total_ganancias.csv',
 'ci_cp_total_horas.csv',
 'ci_rs_total_cursos.csv',
 'ci_rs_volumen_cap.csv',
 'ci_rs_volumen_ganancias.csv',
 'ci_rs_volumen_horas.csv',
 'oc_cp_num_cap_cer.csv',
 'oc_cp_porcentaje_asistencia.csv',
 'oc_cp_total_ganancias.csv',
 'oc_cp_total_horas.csv',
 'oc_rs_total_cursos.csv',
 'oc_rs_volumen_cap_cer.csv',
 'oc_rs_volumen_ganancias.csv',
 'oc_rs_volumen_horas.csv',
 'oc_rs_volumen_porcentaje_asistencia.csv',
 'oec_cp_num_cer.csv',
 'oec_cp_porcentaje_asistencia.csv',
 'oec_cp_total_ganancias.csv',
 'oec_cp_total_horas.csv',
 'oec_rs_total_perfiles.csv',
 'oec_rs_volumen_cer.csv',
 'oec_rs_volumen_ganancias.csv',
 'oec_rs_volumen_horas.csv',
 'oec_rs_volumen_porcentaje_asistencia.csv']

In [30]:
df = []
fact_oc_cp = None
fact_oec_cp = None
fact_ci_cp = None
fact_oc_rs = None
fact_oec_rs = None
fact_ci_rs = None
for file in filelist:
    if '.csv' in file:
            
        df = pd.read_csv(os.path.join(path_model,file))
        idx_pred = [ i for i, word in enumerate(df.columns) if word.startswith('prediction') ]
        pred_col = df.columns[idx_pred][0]
        if 'oc_cp_' in file:
            if fact_oc_cp is None:
                fact_oc_cp = df
            else: 
                fact_oc_cp = fact_oc_cp.merge(df[['id_fact_oc_cp',pred_col]],on='id_fact_oc_cp',how='inner')

        elif 'oec_cp_' in file:
            if fact_oec_cp is None:
                fact_oec_cp = df
            else:
                fact_oec_cp = fact_oec_cp.merge(df[['id_fact_oec_cp',pred_col]],on='id_fact_oec_cp',how='inner')

        elif 'ci_cp_' in file:
            if fact_ci_cp is None:
                fact_ci_cp = df
            else:
                fact_ci_cp = fact_ci_cp.merge(df[['id_fact_ci_cp',pred_col]],on='id_fact_ci_cp',how='inner')

        elif 'oc_rs_' in file:
            if fact_oc_rs is None:
                fact_oc_rs = df
            else:
                fact_oc_rs = fact_oc_rs.merge(df[['id_fact_oc_rs',pred_col]],on='id_fact_oc_rs',how='inner')

        elif 'oec_rs_' in file:
            if fact_oec_rs is None:
                fact_oec_rs = df
            else:
                fact_oec_rs = fact_oec_rs.merge(df[['id_fact_oec_rs',pred_col]],on='id_fact_oec_rs',how='inner')

        elif 'ci_rs_' in file:
            if fact_ci_rs is None:
                fact_ci_rs = df
            else:
                fact_ci_rs = fact_ci_rs.merge(df[['id_fact_ci_rs',pred_col]],on='id_fact_ci_rs',how='inner')


(1107, 26) (1107, 25)
(1107, 27) (1107, 25)
(627, 19) (627, 18)
(627, 20) (627, 18)
(627, 21) (627, 18)
(1932, 27) (1932, 26)
(1932, 28) (1932, 26)
(1932, 29) (1932, 26)
(283, 20) (283, 20)
(283, 21) (283, 20)
(283, 22) (283, 20)
(283, 23) (283, 20)
(18211, 27) (18211, 26)
(18211, 28) (18211, 26)
(18211, 29) (18211, 26)
(12730, 20) (12730, 19)
(12730, 21) (12730, 19)
(12730, 22) (12730, 19)
(12730, 23) (12730, 19)


In [50]:
fact_oc_cp_final = fact_oc_cp[['id_fact_oc_cp','id_razon_social','id_curso_perfil','id_fecha','num_cap_cer','prediction(num_cap_cer)','total_horas','prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)', 'porcentaje_asistencia','prediction(porcentaje_asistencia)']]
fact_oc_rs_final = fact_oc_rs[['id_fact_oc_rs','id_razon_social','id_fecha','volumen_cap_cer','prediction(volumen_cap_cer)','volumen_horas','prediction(volumen_horas)', 'volumen_ganancias', 'prediction(volumen_ganancias)', 'volumen_porcentaje_asistencia','prediction(volumen_porcentaje_asistencia)']]
fact_oec_cp_final = fact_oec_cp[['id_fact_oec_cp','id_razon_social','id_curso_perfil','id_fecha','num_cer','prediction(num_cer)','total_horas','prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)', 'porcentaje_asistencia','prediction(porcentaje_asistencia)']]
fact_oec_rs_final = fact_oec_rs[['id_fact_oec_rs','id_razon_social','id_fecha','volumen_cer','prediction(volumen_cer)','volumen_horas','prediction(volumen_horas)', 'volumen_ganancias', 'prediction(volumen_ganancias)', 'volumen_porcentaje_asistencia','prediction(volumen_porcentaje_asistencia)']]
fact_ci_cp_final = fact_ci_cp[['id_fact_ci_cp','id_razon_social','id_curso_perfil','id_fecha','num_cap','prediction(num_cap)','total_horas','prediction(total_horas)', 'total_ganancias', 'prediction(total_ganancias)']]
fact_ci_rs_final = fact_ci_rs[['id_fact_ci_rs','id_razon_social','id_fecha','volumen_cap','prediction(volumen_cap)','volumen_horas','prediction(volumen_horas)', 'volumen_ganancias', 'prediction(volumen_ganancias)']]

In [75]:
arr_df_finals = [fact_oc_cp_final,fact_oc_rs_final, fact_oec_cp_final,fact_oec_rs_final,fact_ci_cp_final,fact_ci_rs_final]
arr_df_finals_refact = []

In [76]:
for df_final in arr_df_finals:
    df_final_aux = df_final.copy()
    idxs_pred = [ i for i, word in enumerate(df_final_aux.columns) if word.startswith('prediction')]
    for i in idxs_pred:
        old_col = df_final_aux.columns[i]
        new_col = old_col.replace('(','_').replace(')','')
        df_final_aux = df_final_aux.rename(columns={old_col:new_col})
    arr_df_finals_refact.append(df_final_aux)
    
    

In [79]:
for i in range(len(arr_df_finals_refact)):
    df_dpt = arr_df_finals_refact[i]
    doc_name = doc_names[i]
    df_dpt.to_csv(os.path.join(path_dwh_pd,doc_name+'_dpt.csv'),index=False)

# Cargar Datos

In [136]:
arr_tablas = []
file_list_dp = os.listdir(path_dwh_pd)
for file in file_list_dp:
    arr_tablas.append(pd.read_csv(os.path.join(path_dwh_pd,file)))

In [137]:
file_list_dp

['dim_curso_perfil.csv',
 'dim_fecha.csv',
 'dim_razon_social.csv',
 'fact_ci_cp_dpt.csv',
 'fact_ci_rs_dpt.csv',
 'fact_oc_cp_dpt.csv',
 'fact_oc_rs_dpt.csv',
 'fact_oec_cp_dpt.csv',
 'fact_oec_rs_dpt.csv']

In [138]:
df_ert = arr_tablas[0]

In [139]:
df_ert['area_familia']=df_ert['area_familia'].fillna('sin_area_familia')
df_ert['especialidad_sector']=df_ert['especialidad_sector'].fillna('sin_especialidad_sector')
df_ert['curso_perfil']=df_ert['curso_perfil'].fillna('curso_perfil')

In [140]:
df_ert.columns

Index(['id_curso_perfil', 'tipo', 'area_familia', 'especialidad_sector',
       'curso_perfil', 'modalidad', 'carga_horaria', 'costo'],
      dtype='object')

In [141]:
len(df_ert.values)

11149

In [152]:
arr_tablas[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2257 entries, 0 to 2256
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_razon_social    2257 non-null   int64 
 1   razon_social       2257 non-null   object
 2   estado             2257 non-null   object
 3   tipo_razon_social  2257 non-null   object
 4   fecha_resolucion   2257 non-null   object
 5   canton             2257 non-null   object
 6   provincia          2257 non-null   object
dtypes: int64(1), object(6)
memory usage: 123.6+ KB


# Conexión con MySQL

In [142]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='KappaTao1233'
)

mycursor = mydb.cursor()

# Creación de la base de datos¶

In [143]:
mycursor.execute("DROP DATABASE IF EXISTS dwh_setec")
mycursor.execute("CREATE DATABASE dwh_setec")
mycursor.execute("USE dwh_setec")

## DIMENSIÓN CURSO PERFIL

In [144]:
mycursor.execute("DROP TABLE IF EXISTS dim_curso_perfil")
mycursor.execute('''CREATE TABLE dim_curso_perfil (
id_curso_perfil INT PRIMARY KEY,
tipo VARCHAR(63) NOT NULL,
area_familia VARCHAR(255) NOT NULL,
especialidad_sector VARCHAR(255) NOT NULL,
curso_perfil VARCHAR(255) NOT NULL,
modalidad VARCHAR(255) NOT NULL,
carga_horaria INT NOT NULL,
costo FLOAT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_curso_perfil ( id_curso_perfil, tipo, area_familia, especialidad_sector, curso_perfil, modalidad, carga_horaria, costo) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(df_ert.values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|████████████████████████████████████████████████████████████████████████████| 11149/11149 [18:49<00:00,  9.87it/s]


## DIMENSION FECHA

In [150]:
mycursor.execute("DROP TABLE IF EXISTS dim_fecha")
mycursor.execute('''CREATE TABLE dim_fecha (
id_fecha INT PRIMARY KEY,
fecha VARCHAR(63) NOT NULL,
dia INT NOT NULL,
mes INT NOT NULL,
anio INT NOT NULL
)''')
sql_formula = 'INSERT INTO dim_fecha (id_fecha,fecha, dia, mes, anio) VALUES (%s,%s,%s,%s,%s)'
for row in tqdm(arr_tablas[1].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 2142/2142 [01:02<00:00, 34.26it/s]


## DIMENSIÓN RAZÓN SOCIAL

In [153]:
mycursor.execute("DROP TABLE IF EXISTS dim_razon_social")
mycursor.execute('''CREATE TABLE dim_razon_social (
id_razon_social INT PRIMARY KEY,
razon_social VARCHAR(255) NOT NULL,
tipo_razon_social VARCHAR(255) NOT NULL,
estado VARCHAR(63) NOT NULL,
fecha_resolucion VARCHAR(63) NOT NULL,
canton VARCHAR(63) NOT NULL,
provincia VARCHAR(63) NOT NULL
)''')
sql_formula = 'INSERT INTO dim_razon_social ( id_razon_social, razon_social, tipo_razon_social, estado, fecha_resolucion, canton, provincia) VALUES (%s,%s,%s,%s,%s,%s,%s)'
for row in tqdm(arr_tablas[2].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 2257/2257 [01:26<00:00, 26.22it/s]


# DATAMART CI

### CI CP

In [170]:
mycursor.execute("DROP TABLE IF EXISTS fact_ci_cp")
mycursor.execute('''CREATE TABLE fact_ci_cp (
id_fact_ci_cp INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_curso_perfil INT NOT NULL,
id_fecha INT NOT NULL,
num_cap INT NOT NULL,
prediction_num_cap INT NOT NULL,
total_horas INT NOT NULL,
prediction_total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL,
prediction_total_ganancias FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_ci_cp (id_fact_ci_cp,id_razon_social,id_curso_perfil,id_fecha,num_cap,prediction_num_cap,total_horas,prediction_total_horas,total_ganancias,prediction_total_ganancias) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[3].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 1107/1107 [00:31<00:00, 34.99it/s]


### CI RS

In [171]:
mycursor.execute("DROP TABLE IF EXISTS fact_ci_rs")
mycursor.execute('''CREATE TABLE fact_ci_rs (
id_fact_ci_rs INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_fecha INT NOT NULL,
volumen_cap INT NOT NULL,
prediction_volumen_cap INT NOT NULL,
volumen_horas INT NOT NULL,
prediction_volumen_horas INT NOT NULL,
volumen_ganancias FLOAT NOT NULL,
prediction_volumen_ganancias FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_ci_rs (id_fact_ci_rs,
id_razon_social,
id_fecha,
volumen_cap,
prediction_volumen_cap,
volumen_horas,
prediction_volumen_horas,
volumen_ganancias,
prediction_volumen_ganancias)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[4].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|████████████████████████████████████████████████████████████████████████████████| 627/627 [00:29<00:00, 21.58it/s]


## OC

### OC CP

In [172]:
mycursor.execute("DROP TABLE IF EXISTS fact_oc_cp")
mycursor.execute('''CREATE TABLE fact_oc_cp (
id_fact_oc_cp INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_curso_perfil INT NOT NULL,
id_fecha INT NOT NULL,
num_cap_cer INT NOT NULL,
prediction_num_cap_cer INT NOT NULL,
total_horas INT NOT NULL,
prediction_total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL,
prediction_total_ganancias FLOAT NOT NULL,
porcentaje_asistencia FLOAT NOT NULL,
prediction_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oc_cp (id_fact_oc_cp,
id_razon_social,
id_curso_perfil,
id_fecha,
num_cap_cer,
prediction_num_cap_cer,
total_horas,
prediction_total_horas,
total_ganancias,
prediction_total_ganancias,
porcentaje_asistencia,
prediction_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[5].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|██████████████████████████████████████████████████████████████████████████████| 1932/1932 [00:58<00:00, 33.02it/s]


## OC RS

In [177]:
mycursor.execute("DROP TABLE IF EXISTS fact_oc_rs")
mycursor.execute('''CREATE TABLE fact_oc_rs (
id_fact_oc_rs INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_fecha INT NOT NULL,
volumen_cap_cer INT NOT NULL,
prediction_volumen_cap_cer INT NOT NULL,
volumen_horas INT NOT NULL,
prediction_volumen_horas INT NOT NULL,
volumen_ganancias FLOAT NOT NULL,
prediction_volumen_ganancias FLOAT NOT NULL,
volumen_porcentaje_asistencia FLOAT NOT NULL,
prediction_volumen_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oc_rs (id_fact_oc_rs,
id_razon_social,
id_fecha,
volumen_cap_cer,
prediction_volumen_cap_cer,
volumen_horas,
prediction_volumen_horas,
volumen_ganancias,
prediction_volumen_ganancias,
volumen_porcentaje_asistencia,
prediction_volumen_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[6].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|████████████████████████████████████████████████████████████████████████████████| 283/283 [00:10<00:00, 26.07it/s]


## OEC

### OEC CP

In [179]:
mycursor.execute("DROP TABLE IF EXISTS fact_oec_cp")
mycursor.execute('''CREATE TABLE fact_oec_cp (
id_fact_oec_cp INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_curso_perfil INT NOT NULL,
id_fecha INT NOT NULL,
num_cer INT NOT NULL,
prediction_num_cer INT NOT NULL,
total_horas INT NOT NULL,
prediction_total_horas INT NOT NULL,
total_ganancias FLOAT NOT NULL,
prediction_total_ganancias FLOAT NOT NULL,
porcentaje_asistencia FLOAT NOT NULL,
prediction_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oec_cp (id_fact_oec_cp,
id_razon_social,
id_curso_perfil,
id_fecha,
num_cer,
prediction_num_cer,
total_horas,
prediction_total_horas,
total_ganancias,
prediction_total_ganancias,
porcentaje_asistencia,
prediction_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[7].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|████████████████████████████████████████████████████████████████████████████| 18211/18211 [18:48<00:00, 16.13it/s]


### OEC RS

In [185]:
for val in arr_tablas[8].columns:
    print(val)

id_fact_oec_rs
id_razon_social
id_fecha
volumen_cer
prediction_volumen_cer
volumen_horas
prediction_volumen_horas
volumen_ganancias
prediction_volumen_ganancias
volumen_porcentaje_asistencia
prediction_volumen_porcentaje_asistencia


In [187]:
mycursor.execute("DROP TABLE IF EXISTS fact_oec_rs")
mycursor.execute('''CREATE TABLE fact_oec_rs (
id_fact_oec_rs INT PRIMARY KEY,
id_razon_social INT NOT NULL,
id_fecha INT NOT NULL,
volumen_cer INT NOT NULL,
prediction_volumen_cer INT NOT NULL,
volumen_horas INT NOT NULL,
prediction_volumen_horas INT NOT NULL,
volumen_ganancias FLOAT NOT NULL,
prediction_volumen_ganancias FLOAT NOT NULL,
volumen_porcentaje_asistencia FLOAT NOT NULL,
prediction_volumen_porcentaje_asistencia FLOAT NOT NULL
)''')
sql_formula = '''INSERT INTO fact_oec_rs (id_fact_oec_rs,
id_razon_social,
id_fecha,
volumen_cer,
prediction_volumen_cer,
volumen_horas,
prediction_volumen_horas,
volumen_ganancias,
prediction_volumen_ganancias,
volumen_porcentaje_asistencia,
prediction_volumen_porcentaje_asistencia)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
for row in tqdm(arr_tablas[8].values):
    mycursor.execute(sql_formula,tuple(row))
    mydb.commit()

100%|████████████████████████████████████████████████████████████████████████████| 12730/12730 [08:45<00:00, 24.25it/s]


## CREAR CLAVES FORANEAS OC

### FK dim fecha

In [181]:
mycursor.execute(''' 
ALTER TABLE fact_oc_cp
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')

In [188]:
mycursor.execute(''' 
ALTER TABLE fact_oc_rs
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')

### FK dim razon social

In [193]:
mycursor.execute(''' 
ALTER TABLE fact_oc_cp
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')

In [194]:
mycursor.execute(''' 
ALTER TABLE fact_oc_rs
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')

### FK dim curso perfil

In [200]:
mycursor.execute(''' 
ALTER TABLE fact_oc_cp
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil(id_curso_perfil)
''')

## CREAR CLAVES FORANEAS OEC

### FK dim fecha

In [189]:
mycursor.execute(''' 
ALTER TABLE fact_oec_cp
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')

In [190]:
mycursor.execute(''' 
ALTER TABLE fact_oec_rs
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')

### FK dim razon social

In [195]:
mycursor.execute(''' 
ALTER TABLE fact_oec_cp
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')

In [197]:
mycursor.execute(''' 
ALTER TABLE fact_oec_rs
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')

### FK dim curso perfil

In [201]:
mycursor.execute(''' 
ALTER TABLE fact_oec_cp
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil(id_curso_perfil)
''')

## CREAR CLAVES FORANEAS CI

### FK dim fecha

In [191]:
mycursor.execute(''' 
ALTER TABLE fact_ci_cp
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')

In [192]:
mycursor.execute(''' 
ALTER TABLE fact_ci_rs
ADD FOREIGN KEY (id_fecha) REFERENCES dim_fecha(id_fecha)
''')

### FK dim razon social

In [198]:
mycursor.execute(''' 
ALTER TABLE fact_ci_cp
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')

In [199]:
mycursor.execute(''' 
ALTER TABLE fact_ci_rs
ADD FOREIGN KEY (id_razon_social) REFERENCES dim_razon_social(id_razon_social)
''')

### FK dim curso perfil

In [202]:
mycursor.execute(''' 
ALTER TABLE fact_ci_cp
ADD FOREIGN KEY (id_curso_perfil) REFERENCES dim_curso_perfil(id_curso_perfil)
''')